## Importar Datos desde Sitio Web de la ANH (Datos Oficiales)

In [1]:
import os
import pandas as pd
import re
import requests
import numpy as np
import matplotlib.pyplot as plt
import time
from multiprocessing.pool import ThreadPool
from progress.bar import ChargingBar

#*******************************************************************************************
# Crear Clase para Importar datos desde pagina web de la ANH
class Importar_Datos_Oficiales:


    def __init__(self):

        BASE_URL = 'http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones' #direccion url de la ANH
        # Url's de los archivos de produccion de crudo
        file2018_url = '/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202018.xlsx' 
        file2019_url = '/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202019-DIC.xlsx'
        file2020_url = '/Documentos%20compartidos/Producci%c3%b3n%20Fiscalizada%20Crudo%202020%20Agosto.xlsx'
        
        self.URLS = {'Produccion_Fiscalizada_Crudo_2018': f'{BASE_URL}'+file2018_url,
                'Produccion_Fiscalizada_Crudo_2019': f'{BASE_URL}'+file2019_url,
                'Produccion_Fiscalizada_Crudo_2020':f'{BASE_URL}'+file2020_url, 
        }


        self.data = {case:pd.read_excel(url) for case, url in self.URLS.items()}
#*******************************************************************************************

# Obtener datos desde la pagina web de la ANH
datos_importados = Importar_Datos_Oficiales()

# Guardar datos importados en DataFrames para posterior procesamiento
df_2018 = datos_importados.data['Produccion_Fiscalizada_Crudo_2018']
df_2019 = datos_importados.data['Produccion_Fiscalizada_Crudo_2019']
df_2020 = datos_importados.data['Produccion_Fiscalizada_Crudo_2020']

In [2]:
# Funcion para limpieza del archivo excel y poner en formato
def format_excel(df_excel_raw, year = 2020):
    df_excel_raw.dropna(inplace = True) #eliminar filas NO validas
    df_excel_raw.rename(columns=df_excel_raw.iloc[0],inplace = True) #reemplazar el nombre de las columnas
    df_excel_raw.drop(df_excel_raw.index[0], inplace = True)
    df_excel_raw.reset_index(drop = True, inplace = True) # reiniciar el indexado de las filas
    df_excel_raw.columns = df_excel_raw.columns.str.strip().str.lower() # convertir nombre de columnas a minusculas y sin espacios
    
    meses = ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
        
    for mes in meses:
        if (0 in np.core.defchararray.find( df_excel_raw.columns.values.astype(str), mes)):
            # reemplazar nombre de Mes a nombre corto (Enero --> ene)
            df_excel_raw.columns.values[[i for i,item in enumerate(df_excel_raw.columns.values) if mes in item]] = mes            

        else:
            # Si no hay datos para el mes, anadir una columna de ceros para ese mes
            df_excel_raw[mes] = pd.Series([0 for x in range(len(df_excel_raw.index))], index=df_excel_raw.index)
            
    # anadir columna con año para identificacion
    df_excel_raw['año'] = pd.Series([year for x in range(len(df_excel_raw.index))], index=df_excel_raw.index)
    
    return df_excel_raw

In [3]:
format_excel(df_2020,2020)
format_excel(df_2019,2019)
format_excel(df_2018,2018)
df_2020.head(5) #mostrar produccion de crudo - 2020

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949,0,0,0,0,2020
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08,0,0,0,0,2020
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73,0,0,0,0,2020
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94,0,0,0,0,2020
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,187.33,184.97,173.58,191.32,193.94,166.74,168.84,152.02,0,0,0,0,2020


In [4]:
# Unir produccion desde 2018 a 2020 en Un unico DataFrame
df_2018_2020 = pd.concat([df_2020, df_2019, df_2018], ignore_index = True)
df_2018_2020.head()

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949,0,0,0,0,2020
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08,0,0,0,0,2020
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73,0,0,0,0,2020
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94,0,0,0,0,2020
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,187.33,184.97,173.58,191.32,193.94,166.74,168.84,152.02,0,0,0,0,2020


In [5]:
# convertir datos de produccion a formato numerico
for mes in ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']:
    df_2018_2020[mes] = pd.to_numeric(df_2018_2020[mes])

In [6]:
# Formato LONG: Datos de Produccion de CRUDO (2018 a 2020)
df_2018_2020_LONG = pd.melt(df_2018_2020, id_vars=['departamento','municipio','operadora','contrato','campo','año'], var_name = 'mes', value_name = 'produccion')
df_2018_2020_LONG.head()

,departamento,municipio,operadora,contrato,campo,año,mes,produccion
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,2020,ene,1765.84
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,2020,ene,119.70
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,2020,ene,11636.58
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,2020,ene,1852.60
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,2020,ene,187.33


In [7]:
# Archivo de Salida (Output): Datos de Produccion de Crudo (2018 a 2020):

# Crear Carpeta 'Datos_salida' donde guardar archivos de salida
path = os.getcwd()
print(path)
dir = os.path.join(path,"Datos_salida")
if not os.path.exists(dir):
    os.mkdir(dir)

    
# Exportar a .xlsx - produccion 2018 a 2020
df_2018_2020_LONG.to_excel('Datos_salida/Produccion_Fiscalizada_Crudo_DatosOficiales_2018-2020.xlsx', index = False)

C:\Users\jnina\Jupyter\HackathonOG


## Importar Datos No Oficiales desde Repositorio Hacathon O&G (GitHub)

In [ ]:
#*******************************************************************************************
# Crear Clase para Importar datos blind desde Repositorio Hackathon O&G 
class Importar_Datos_Blind:


    def __init__(self):

        BASE_URL = 'https://github.com/specolombiahackathon/202010' #url del repositorio (incluir raw) con datos blind
        # url's de los datos de produccion (blind)
        file2018_url = 'path_file_2018' 
        file2019_url = 'path_file_2019'
        file2020_url = 'path_file_2020'
        
        self.URLS = {'Produccion_Fiscalizada_Crudo_2018_blind': f'{BASE_URL}'+file2018_url,
                'Produccion_Fiscalizada_Crudo_2019_blind': f'{BASE_URL}'+file2019_url,
                'Produccion_Fiscalizada_Crudo_2020_blind':f'{BASE_URL}'+file2020_url, 
        }


        self.data = {case:pd.read_excel(url) for case, url in self.URLS.items()}
#*******************************************************************************************

# Obtener datos desde la pagina web de la ANH
datos_importados_blind = Importar_Datos_Blind()

# Guardar datos importados en DataFrames para posterior procesamiento
df_2018_blind = datos_importados_blind.data['Produccion_Fiscalizada_Crudo_2018']
df_2019_blind = datos_importados_blind.data['Produccion_Fiscalizada_Crudo_2019']
df_2020_blind = datos_importados_blind.data['Produccion_Fiscalizada_Crudo_2020']


In [ ]:
format_excel(df_2020_blind,2020)
format_excel(df_2019_blind,2019)
format_excel(df_2018_blind,2018)
df_2020_blind.head(5) #mostrar produccion de crudo - 2020

In [ ]:
# Unir produccion desde 2018 a 2020 en Un unico DataFrame
df_2018_2020_blind = pd.concat([df_2020_blind, df_2019_blind, df_2018_blind], ignore_index = True)
df_2018_2020_blind.head()

In [ ]:
# convertir datos de produccion a formato numerico
for mes in ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']:
    df_2018_2020_blind[mes] = pd.to_numeric(df_2018_2020_blind[mes])

In [ ]:
# Formato LONG: Datos de Produccion de CRUDO (2018 a 2020)
df_2018_2020_LONG_blind = pd.melt(df_2018_2020, id_vars=['departamento','municipio','operadora','contrato','campo','año'], var_name = 'mes', value_name = 'produccion')
df_2018_2020_LONG_blind.head()

In [ ]:
# Archivo de Salida (Output): Datos de Produccion de Crudo (2018 a 2020):

# Crear Carpeta 'Datos_salida' donde guardar archivos de salida
path = os.getcwd()
print(path)
dir = os.path.join(path,"Datos_salida")
if not os.path.exists(dir):
    os.mkdir(dir)

    
# Exportar a .xlsx - produccion 2018 a 2020
df_2018_2020_LONG_blind.to_excel('Datos_salida/Produccion_Fiscalizada_Crudo_Blind_2018-2020.xlsx', index = False)